# Tutorial 01 - Kuramoto and friends

This notebook explores three core generators:

1. Kuramoto oscillators with alternative topologies
2. Ornstein-Uhlenbeck (OU) processes
3. Gaussian noise

Randomness flows through SeedSequence.spawn(), keeping runs deterministic.

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.use('Agg', force=True)
import matplotlib.pyplot as plt
from pathlib import Path

from pymts import Generator
from pymts.plotting import plot_heatmap, plot_timeseries


## 1. Kuramoto topologies
We compare ring versus complete coupling while keeping other parameters fixed.

In [ ]:
generator = Generator()

ring_cfg = {
    'model': 'kuramoto',
    'M': 5,
    'T': 128,
    'K': 0.6,
    'topology': 'ring',
    'n_realizations': 2,
    'seed': 2024,
}
complete_cfg = {**ring_cfg, 'topology': 'complete'}
ring_ds, complete_ds = generator.generate([ring_cfg, complete_cfg])
ring_ds


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(9, 6), constrained_layout=True)
plot_heatmap(ring_ds, realization=0, ax=axes[0, 0])
plot_timeseries(ring_ds, realization=0, stems=True, ax=axes[0, 1])
plot_heatmap(complete_ds, realization=0, ax=axes[1, 0])
plot_timeseries(complete_ds, realization=0, stems=True, ax=axes[1, 1])
axes[0, 0].set_title('Ring heatmap')
axes[0, 1].set_title('Ring stems')
axes[1, 0].set_title('Complete heatmap')
axes[1, 1].set_title('Complete stems')
plt.show()


## 2. Ornstein-Uhlenbeck
Exact discretisation keeps the simulation stable even for stiff parameters.

In [ ]:
ou_cfg = {
    'model': 'ou',
    'M': 3,
    'T': 96,
    'dt': 0.1,
    'mu': [0.0, 0.5, -0.5],
    'theta': 0.7,
    'sigma': [0.2, 0.3, 0.4],
    'n_realizations': 2,
    'seed': 314,
}
ou_ds = generator.generate([ou_cfg])[0]
ou_ds


In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
plot_timeseries(ou_ds, realization=0, ax=ax)
ax.set_title('OU trajectories (realization 0)')
plt.show()


## 3. Gaussian noise
IID noise is handy for benchmarking feature pipelines.

In [ ]:
noise_cfg = {
    'model': 'noise_gaussian',
    'M': 4,
    'T': 80,
    'sigma': 0.4,
    'n_realizations': 3,
    'seed': 777,
}
noise_ds = generator.generate([noise_cfg])[0]
noise_ds['data'].std(dim='time').values


## 4. Z-score and persistence
Use zscore=True to normalise each channel/realization pair, then persist outputs.

In [ ]:
z_ds = generator.generate([ring_cfg], zscore=True)[0]
z_means = z_ds['data'].mean(dim='time').values.round(3)
z_stds = z_ds['data'].std(dim='time').values.round(3)
z_means, z_stds


In [ ]:
from pymts.io import save_parquet, write_sidecar_metadata
store_dir = Path('data/tutorial') / ring_ds.attrs['config_id']
store_dir.mkdir(parents=True, exist_ok=True)
config_id = ring_ds.attrs['config_id']
save_parquet(ring_ds, store_dir / (config_id + '.parquet'))
write_sidecar_metadata(store_dir / (config_id + '.metadata.json'), ring_ds.attrs)
